<a href="https://colab.research.google.com/github/Dhaneshkp/DesktopAssistant/blob/main/DATA_processing_expConv2Dv4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
from math import exp
import numpy as np

# Set a random seed for reproducibility
np.random.seed(42)

# Parameters
n_samples = 100000
unique_risk_classes = ['A', 'B', 'C', 'D']
n_anomalies = 2000
# Generate random ages, durations, and premiums
ages = np.random.randint(18, 70, size=(n_samples, 1))
durations = np.random.randint(20, 35, size=(n_samples, 1))
premiums = ages * np.exp(durations/20) + np.random.randint(10, 50, size=(n_samples, 1))/2

# Define risk class multipliers
riskclass_multipliers = {
    'A': 1.05,
    'B': 1.1,
    'C': 1.3,
    'D': 1.0
}

# Initialize the result array
result_array = np.empty((n_samples, 1, 3, len(unique_risk_classes)))

# Iterate over samples and risk classes
for i in range(n_samples):
    for j, risk_class in enumerate(unique_risk_classes):
        # Apply risk class multiplier to premium
        premium_with_risk = premiums[i] * riskclass_multipliers[risk_class]

        # Assign age, duration, and premium to the result array
        result_array[i, 0, :, j] = np.concatenate((ages[i], durations[i], premium_with_risk))

anomaly_indices = np.random.choice(n_samples * len(unique_risk_classes), n_anomalies, replace=False)
for idx in anomaly_indices:
    sample_idx = idx // len(unique_risk_classes)
    risk_class_idx = idx % len(unique_risk_classes)
    result_array[sample_idx, 0, 2, risk_class_idx] += np.random.randint(700, 1000)/10  # Add a large random value to the premium

print(result_array.shape)  # Output: (10000, 1, 3, 4)
result_array
#anomaly_indices

(100000, 1, 3, 4)


array([[[[ 56.        ,  56.        ,  56.        ,  56.        ],
         [ 23.        ,  23.        ,  23.        ,  23.        ],
         [194.10174309, 203.34468324, 240.31644383, 184.85880294]]],


       [[[ 69.        ,  69.        ,  69.        ,  69.        ],
         [ 21.        ,  21.        ,  21.        ,  21.        ],
         [217.5368235 , 227.89571986, 269.33130529, 207.17792715]]],


       [[[ 46.        ,  46.        ,  46.        ,  46.        ],
         [ 28.        ,  28.        ,  28.        ,  28.        ],
         [212.6661584 , 222.79311832, 263.30095802, 202.53919847]]],


       ...,


       [[[ 33.        ,  33.        ,  33.        ,  33.        ],
         [ 20.        ,  20.        ,  20.        ,  20.        ],
         [113.08846536, 118.47363037, 140.01429044, 107.70330034]]],


       [[[ 57.        ,  57.        ,  57.        ,  57.        ],
         [ 26.        ,  26.        ,  26.        ,  26.        ],
         [231.68240556, 242.7149

In [96]:
# prompt: reshape into a darat drame

import numpy as np
import pandas as pd

# ... (Preceding code remains the same)

# Reshape the array into a 2D format
reshaped_array = (X_test).reshape(-1,3*4)

# Create a DataFrame
df = pd.DataFrame(reshaped_array)
# Display the DataFrame
df


,0,1,2,3,4,5,6,7,8,9,10,11
0,18.0,18.0,18.0,18.0,24.0,24.0,24.0,24.0,28.4550,29.810,35.230,27.10
1,56.0,56.0,56.0,56.0,34.0,34.0,34.0,34.0,111.5100,116.820,138.060,106.20
2,56.0,56.0,56.0,56.0,32.0,32.0,32.0,32.0,106.6800,111.760,132.080,101.60
3,49.0,49.0,49.0,49.0,29.0,29.0,29.0,29.0,100.3275,105.105,124.215,95.55
4,56.0,56.0,56.0,56.0,22.0,22.0,22.0,22.0,89.8800,94.160,111.280,85.60
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,28.0,28.0,28.0,28.0,21.0,21.0,21.0,21.0,38.2200,40.040,47.320,36.40
19996,57.0,57.0,57.0,57.0,21.0,21.0,21.0,21.0,73.8675,77.385,91.455,70.35
19997,66.0,66.0,66.0,66.0,22.0,22.0,22.0,22.0,87.7800,91.960,108.680,83.60
19998,45.0,45.0,45.0,45.0,32.0,32.0,32.0,32.0,95.0250,99.550,117.650,90.50


In [92]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Conv2DTranspose,Dropout
from tensorflow.keras.regularizers import l2
X_train, X_test, train_indices, test_indices = train_test_split(
    result_array,
    np.arange(result_array.shape[0]),  # Create indices for the array
    test_size=0.2,
    random_state=42
)
#train_indices, test_indices = train_test_split(data_reshaped.index, test_size=0.2, random_state=42)
# Define the CNN architecture
input_shape = (X_train.shape[1], X_train.shape[2],X_train.shape[3])    # Assuming X_train is a 3D array
input_layer = Input(shape=input_shape)
# Define the CNN architecture
input_shape = (X_train.shape[1], X_train.shape[2],X_train.shape[3])    # Assuming X_train is a 3D array
input_layer = Input(shape=input_shape)

conv_layer = Conv2D(4, (1,3), activation="relu",padding='same',kernel_regularizer=l2(0.01))(input_layer)
conv_layer = MaxPooling2D(pool_size=(1,1))(conv_layer)
dropout_layer = Dropout(0.2)(conv_layer)
output_shape = (conv_layer.shape[1], conv_layer.shape[2], conv_layer.shape[3])  # Adjust as needed

# Reshape the output of the convolutional layers
reshape_layer = Reshape(output_shape)(conv_layer)

# Use transposed convolution (Conv2DTranspose) to upsample and reconstruct
deconv_layer = Conv2DTranspose(4, (1, 3), activation='relu', padding='same')(reshape_layer)
output_layer = Conv2D(X_train.shape[3], (1, 1), activation='linear', padding='same')(deconv_layer)  # Output 4 channels

cnn_model = Model(inputs=input_layer, outputs=output_layer)
cnn_model.compile(optimizer='adam', loss='mse')

# ... (rest of your code for training and prediction)

cnn_model = Model(inputs=input_layer, outputs=output_layer)
cnn_model.compile(optimizer='adam', loss='mse')

# Train the model
history = cnn_model.fit(X_train, X_train,
                        epochs=25,
                        batch_size=64,
                        validation_split=0.1,
                        verbose=1)

# Predict the training data
pred_train = cnn_model.predict(X_train)
mse_train = np.mean(np.power(X_train - pred_train,2), axis=1)



Epoch 1/25
1125/1125 [==============================] - 2s 2ms/step - loss: 206.5519 - val_loss: 16.0886
Epoch 2/25
1125/1125 [==============================] - 2s 2ms/step - loss: 13.6011 - val_loss: 13.4017
Epoch 3/25
1125/1125 [==============================] - 1s 1ms/step - loss: 11.3062 - val_loss: 11.4288
Epoch 4/25
1125/1125 [==============================] - 1s 1ms/step - loss: 9.7196 - val_loss: 10.2157
Epoch 5/25
1125/1125 [==============================] - 1s 1ms/step - loss: 8.6584 - val_loss: 9.2292
Epoch 6/25
1125/1125 [==============================] - 2s 1ms/step - loss: 7.6773 - val_loss: 8.2141
Epoch 7/25
1125/1125 [==============================] - 1s 1ms/step - loss: 6.7867 - val_loss: 7.5213
Epoch 8/25
1125/1125 [==============================] - 1s 1ms/step - loss: 6.3581 - val_loss: 7.2841
Epoch 9/25
1125/1125 [==============================] - 2s 2ms/step - loss: 6.2310 - val_loss: 7.2046
Epoch 10/25
1125/1125 [==============================] - 2s 1ms/step - los

In [98]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Reshape

model = Sequential()
model.add(Conv2D(32, (1, 3), activation='relu', input_shape=X_train.shape[1:]))  # Adjusted kernel size
model.add(Flatten())
model.add(Dense(64, activation='relu'))  # Bottleneck layer
model.add(Dense(np.prod(X_train.shape[1:]), activation='linear')) # Output layer for reconstruction
model.add(Reshape(X_train.shape[1:])) # Reshape to original input shape

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, X_train, epochs=10)  # Train on normal data

# Anomaly Detection
reconstructions = model.predict(X_test)
mse = np.mean(np.square(X_test - reconstructions), axis=(1, 2, 3))

Epoch 1/10
2500/2500 [==============================] - 4s 2ms/step - loss: 35.9978
Epoch 2/10
2500/2500 [==============================] - 3s 1ms/step - loss: 0.0331
Epoch 3/10
2500/2500 [==============================] - 3s 1ms/step - loss: 0.0275
Epoch 4/10
2500/2500 [==============================] - 3s 1ms/step - loss: 0.0263
Epoch 5/10
2500/2500 [==============================] - 3s 1ms/step - loss: 0.0260
Epoch 6/10
2500/2500 [==============================] - 3s 1ms/step - loss: 0.0203
Epoch 7/10
2500/2500 [==============================] - 3s 1ms/step - loss: 0.0243
Epoch 8/10
2500/2500 [==============================] - 3s 1ms/step - loss: 0.0183
Epoch 9/10
2500/2500 [==============================] - 3s 1ms/step - loss: 0.0197
Epoch 10/10
625/625 [==============================] - 1s 850us/step


In [99]:
msethreshold = np.percentile(mse_train, 99)

# Identify anomalies in the test set
anomalies = mse_test > threshold

# Print the results
print(f"Threshold: {threshold}")
print(f"Number of anomalies in the test set: {np.sum(anomalies)}")

print(X_test)
print(pred_test)
print(threshold)

array([0.00044168, 0.0024787 , 0.002276  , ..., 0.00161858, 0.00188698,
       0.04001099])

In [88]:
cnn_model.summary()

Model: "model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 1, 3, 4)]         0         
                                                                 
 conv2d_18 (Conv2D)          (None, 1, 3, 4)           52        
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 1, 3, 4)           0         
 g2D)                                                            
                                                                 
 reshape_9 (Reshape)         (None, 1, 3, 4)           0         
                                                                 
 conv2d_transpose_9 (Conv2D  (None, 1, 3, 4)           52        
 Transpose)                                                      
                                                                 
 conv2d_19 (Conv2D)          (None, 1, 3, 4)           20 

In [93]:

# Predict the test data
pred_test = cnn_model.predict(X_test)
mse_test = np.mean(np.power(X_test - pred_test  , 2), axis=1)

# Define a threshold for anomaly detection
threshold = np.percentile(mse_train, 99)

# Identify anomalies in the test set
anomalies = mse_test > threshold

# Print the results
print(f"Threshold: {threshold}")
print(f"Number of anomalies in the test set: {np.sum(anomalies)}")

print(X_test)
print(pred_test)
print(threshold)

625/625 [==============================] - 1s 901us/step
Threshold: 0.2304802804294941
Number of anomalies in the test set: 2375
[[[[ 18.     18.     18.     18.   ]
   [ 24.     24.     24.     24.   ]
   [ 28.455  29.81   35.23   27.1  ]]]


 [[[ 56.     56.     56.     56.   ]
   [ 34.     34.     34.     34.   ]
   [111.51  116.82  138.06  106.2  ]]]


 [[[ 56.     56.     56.     56.   ]
   [ 32.     32.     32.     32.   ]
   [106.68  111.76  132.08  101.6  ]]]


 ...


 [[[ 66.     66.     66.     66.   ]
   [ 22.     22.     22.     22.   ]
   [ 87.78   91.96  108.68   83.6  ]]]


 [[[ 45.     45.     45.     45.   ]
   [ 32.     32.     32.     32.   ]
   [ 95.025  99.55  117.65   90.5  ]]]


 [[[ 68.     68.     68.     68.   ]
   [ 30.     30.     30.     30.   ]
   [128.625 134.75  159.25  200.8  ]]]]
[[[[ 17.964975  18.00278   18.007193  17.984373]
   [ 23.9559    24.070087  24.005219  23.991842]
   [ 28.445753  29.833885  35.25842   27.141346]]]


 [[[ 55.819527  56.35147

In [81]:
# Find indices of anomalies in X_test
anomaly_indices = np.where(anomalies)[0]

# Extract the anomalous records from X_test
anomalous_records = X_test[anomaly_indices]

print("Anomalous Records:")
print(anomalous_records)
anomaly_indices

Anomalous Records:
[[[[  69.     69.     69.     69.  ]
   [   6.      6.      6.      6.  ]
   [ 479.85  502.7   594.1   457.  ]]]


 [[[  68.     68.     68.     68.  ]
   [  16.     16.     16.     16.  ]
   [1176.   1232.   1456.   1120.  ]]]


 [[[  67.     67.     67.     67.  ]
   [   2.      2.      2.      2.  ]
   [ 172.2   180.4   213.2   164.  ]]]


 ...


 [[[  39.     39.     39.     39.  ]
   [  18.     18.     18.     18.  ]
   [1553.8   787.6   930.8   716.  ]]]


 [[[  39.     39.     39.     39.  ]
   [  18.     18.     18.     18.  ]
   [1553.8   787.6   930.8   716.  ]]]


 [[[  68.     68.     68.     68.  ]
   [  15.     15.     15.     15.  ]
   [1121.4  1174.8  1388.4  1068.  ]]]]


array([    6,    14,    18, ..., 19990, 19990, 19999])

In [23]:
# Create a copy of the original test data from the NumPy array
test_indices
anomalies


array([[[False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]],

       [[False, False, False, False],
        [False, False, False, False],
        [False,  True, False, False]],

       [[False, False, False, False],
        [False, False, False, False],
        [False,  True, False, False]],

       ...,

       [[False, False, False, False],
        [False, False, False, False],
        [False,  True, False, False]],

       [[False, False, False, False],
        [False, False, False, False],
        [False,  True, False, False]],

       [[False, False, False, False],
        [False, False, False, False],
        [ True,  True, False, False]]])

In [ ]:
# prompt: crate an isolation forest to detect anomalies in teh result_df_encoded and fina the index of anomalies in teh data and crate a data frame like test_df_with_anomalies , use the xtrain and xtest etc using the xtrain and xtest indices created in the previuos code, and for isolation forest i think we need y test and train so creat theam , if we drop premium rate then thas sdropiing the ioput

import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split

# Assuming 'result_df_encoded' is your DataFrame from the preceding code

# Prepare the data for Isolation Forest
X = result_df_encoded.drop('Premium_rate', axis=1).values  # Assuming 'Premium_rate' is the target variable

# Split the data into training and test sets (using the same indices as before)
X_train = X[train_indices]
X_test = X[test_indices]
y_train = result_df_encoded.loc[train_indices, 'Premium_rate'].values
y_test = result_df_encoded.loc[test_indices, 'Premium_rate'].values

# Create an Isolation Forest model
model = IsolationForest(contamination=0.05, random_state=42)  # Adjust contamination as needed

# Fit the model to the training data
model.fit(X_train)

# Predict anomalies on the test data
y_pred = model.predict(X_test)

# Identify anomalies (-1 indicates an anomaly)
anomalies = y_pred == -1

# Create a copy of the original test DataFrame
test_df_with_anomalies_if = result_df_encoded.iloc[test_indices].copy()

# Add an 'anomaly' column to the test DataFrame
test_df_with_anomalies_if['anomaly'] = anomalies

# Print the results
print(f"Number of anomalies in the test set (Isolation Forest): {np.sum(anomalies)}")

# Display the test DataFrame with anomaly tags
test_df_with_anomalies_if


Number of anomalies in the test set (Isolation Forest): 455


,Age,Duration,Premium_rate,level_2_Female_NP2,level_2_Female_NS2,level_2_Male_NP2,level_2_Male_NS2,anomaly
21185,22.0,4.0,8.359788,0.0,0.0,1.0,0.0,False
21081,41.0,2.0,4.408889,1.0,0.0,0.0,0.0,False
38072,35.0,9.0,9.813333,0.0,0.0,0.0,1.0,False
37127,21.0,9.0,7.733333,0.0,0.0,0.0,1.0,False
22786,38.0,3.0,5.911330,0.0,0.0,0.0,1.0,False
...,...,...,...,...,...,...,...,...
25543,42.0,3.0,8.059259,0.0,0.0,1.0,0.0,False
23315,28.0,5.0,7.714286,0.0,0.0,1.0,0.0,False
8920,41.0,7.0,12.370370,0.0,0.0,1.0,0.0,False
21822,47.0,5.0,15.828571,0.0,0.0,1.0,0.0,False
